# Registering Your Optimized Model to Azure ML

This notebook demonstrates how to register your fine-tuned and optimized model to the Azure Machine Learning registry. Model registration is crucial for version tracking, sharing, and deploying models in a production environment.

![](../../lab_manual/images/step-3.png)

## What You'll Learn

- How to authenticate with Azure Machine Learning
- How to create an ML client to interact with your Azure ML workspace
- How to register model files to the Azure ML registry
- How to add metadata and descriptions to your model
- How to list and verify your registered models

## Prerequisites

- Completed the previous notebooks:
  - `01.AzureML_Distillation.ipynb` (generated training data)
  - `02.AzureML_FineTuningAndConvertByMSOlive.ipynb` (fine-tuned and optimized the model)
  - `03.AzureML_RuningByORTGenAI.ipynb` (tested the optimized model)
- Successfully created model files in `models/phi-4-mini/onnx/`
- Access to an Azure ML workspace
- Python environment with necessary libraries (which we'll install)

## Setup Instructions

1. **Azure Authentication**: Ensure you're logged in to Azure using `az login --use-device-code` in a terminal
2. **Kernel Selection**: Change the Jupyter kernel to **"Python 3.10 AzureML"** using the selector in the top right
3. **Environment File**: Ensure your `local.env` file exists with proper Azure ML workspace information


## Initial Setup

Before proceeding with this notebook, complete these important setup steps:

1. **Azure Authentication**: Run `az login --use-device-code` in a terminal to authenticate with Azure. This will provide the credentials needed to access your Azure ML workspace.

2. **Kernel Selection**: Select the "Python 3.10 AzureML" kernel from the dropdown menu in the top-right corner of this notebook. This kernel has most of the necessary Azure ML libraries pre-installed.

3. **Environment Variables**: Ensure your `local.env` file contains the following variables:
   - AZUREML_SUBSCRIPTION_ID
   - AZUREML_RESOURCE_GROUP
   - AZUREML_WS_NAME

## 1. Install Environment Variable Handling Package

First, we'll install the `python-dotenv` package. This library allows us to load environment variables from a `.env` file, making it easier to handle configuration settings securely.

Environment variables are the recommended way to manage sensitive information like subscription IDs and workspace details, as they keep this information out of your code.

In [2]:
pip install dotenv-azd

Note: you may need to restart the kernel to use updated packages.


## 2. Import Azure ML Model Components

Here we import the basic components needed to define and register a model in Azure ML:

- **Model**: Class from `azure.ai.ml.entities` that represents a machine learning model in Azure ML

- **AssetTypes**: Constants that define the types of assets we can register, such as custom models, datasets, etc.

These components will help us define our model for registration.

In [3]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

## 3. Import Complete Azure ML SDK Components

Now we import all the necessary components from the Azure ML SDK to work with the ML workspace and register our model:

- **MLClient**: The main client for interacting with Azure ML services

- **Input**: Used to define inputs for Azure ML components

- **Model**: As imported previously, for defining our model (repeated import)

- **AssetTypes**: As imported previously, for defining asset types (repeated import)

- **DefaultAzureCredential**: From `azure.identity`, this class provides a default credential flow for authenticating with Azure services

These imports give us everything we need to connect to Azure ML and register our model.

In [4]:
from azure.ai.ml import MLClient, Input
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential

## 4. Import Environment Variable Handling

Here we import the libraries needed to access environment variables:

- **os**: For accessing environment variables and file paths

- **load_azd_env**: From `dotenv_azd`, for loading environment variables from Azure Developer CLI (AZD) environments

- **load_dotenv**: From `dotenv`, for loading environment variables from local `.env` files

This combination allows us to access configuration values from either AZD environments or local `.env` files, giving us flexibility in how we manage our Azure ML workspace credentials.

In [5]:
import os
from dotenv_azd import load_azd_env
from dotenv import load_dotenv

# Load environment variables from current AZD environment if available
load_azd_env(quiet=True)

# Load environment variables from local.env file if it exists
load_dotenv(dotenv_path="local.env")

True

## 5. Load Environment Variables

This cell loads our Azure ML workspace credentials from environment variables. We use a two-step approach:

1. First, try to load variables from an Azure Developer CLI (AZD) environment if available using `load_azd_env()`

2. Then, load any additional or override variables from a local `.env` file using `load_dotenv()`

This approach allows us to work with either AZD environments or local files, providing flexibility in how we manage configuration. 

The environment variables we need are:
- AZUREML_SUBSCRIPTION_ID
- AZUREML_RESOURCE_GROUP
- AZUREML_WS_NAME

## 6. Initialize Azure ML Client

In this step, we retrieve our workspace information from the environment variables and create an ML client to interact with Azure ML:

1. First, we get the necessary Azure ML workspace information from environment variables:
   - `subscription_id`: The Azure subscription ID that contains your workspace
   - `resource_group`: The resource group containing your workspace
   - `workspace`: The name of your Azure ML workspace

2. We print these values to verify them (in a production environment, you might remove this debugging output)

3. Finally, we create an `MLClient` object using:
   - `DefaultAzureCredential()`: Uses the default Azure authentication chain
   - The subscription ID, resource group, and workspace name

This client will be our main interface for interacting with Azure ML and registering our model.

In [6]:
# Get Azure ML credentials from environment variables
subscription_id = os.getenv('AZUREML_SUBSCRIPTION_ID')
resource_group = os.getenv('AZUREML_RESOURCE_GROUP')
workspace = os.getenv('AZUREML_WS_NAME')

# Print values for debugging (remove in production)
print(f"Subscription ID: {subscription_id}")
print(f"Resource Group: {resource_group}")
print(f"Workspace: {workspace}")

# Create ML client with the credentials
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

Subscription ID: 6415ebd4-1dd7-430f-bd4d-2f5e9419c1cd
Resource Group: rg-cvi-lab329-h-3
Workspace: cvi-lab329-h-3


## 7. Define the Model for Registration

Here we create a Model object that defines the details of our model for registration in Azure ML:

- **path**: Points to the folder containing our optimized ONNX model files

- **type**: Specifies that this is a custom model (rather than a standard framework model)

- **name**: The name under which our model will be registered in Azure ML

- **description**: A brief description of our model to help others understand its purpose

These attributes provide important metadata and context for our model in the Azure ML registry. The name should be descriptive and include information about the model type and optimization level.

In [7]:
file_model = Model(
    path="models/phi-4-mini/onnx",
    type=AssetTypes.CUSTOM_MODEL,
    name="fine-tuning-phi-4-mini-onnx-int4-cpu",
    description="Fine tuning by MSOlive",
)

## 8. Register the Model in Azure ML

This cell performs the actual registration of our model to the Azure ML model registry:

1. We use the `ml_client.models.create_or_update()` method to register our model

2. The method takes our model definition (`file_model`) and uploads the files from the specified path to Azure ML

3. If a model with the same name already exists, a new version will be created; if not, version 1 will be created

4. The function returns a reference to the registered model, which we store in `registered_model`

This step may take some time depending on the size of your model files and your internet connection speed.

In [8]:
registered_model = ml_client.models.create_or_update(file_model)

Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy '/afh/projects/cvi-lab329-h-3-d0ca370a-6510-40b5-b1dc-98d97b208684/shared/Users/cedricvidal/models/phi-4-mini/onnx' 'https://sthubvc4zovnn4uqts.blob.core.windows.net/d0ca370a-6510-40b5-b1dc-98d97b208684-azureml-blobstore/LocalUpload/4e85d7df997d513303aa791ce94b3cc5/onnx' 

See https://learn.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.
Uploading onnx (4975.45 MBs): 100%|██████████| 4975453220/4975453220 [00:13<00:00, 366428957.40it/s]




## 9. Verify and Explore the Registered Model

After registration, we can verify our model's details and explore its properties:

1. We print key information about our registered model:
   - Name: The model's name in the registry
   - ID: The unique identifier for the model
   - Version: The version number (increments with each update)
   - Description: The description we provided
   - Path: Where the model is stored in Azure ML

2. We also print all properties of the registered model by iterating through its `__dict__` attribute

This gives us a complete view of the model's metadata and confirms that it was registered correctly. You can now find this model in your Azure ML workspace's Models section.

In [9]:
# Display model details
print(f"Model Name: {registered_model.name}")
print(f"Model ID: {registered_model.id}")
print(f"Model Version: {registered_model.version}")
print(f"Model Description: {registered_model.description}")
print(f"\nModel Location: {registered_model.path}")
print(f"\nFull Model Properties:")
for key, value in registered_model.__dict__.items():
    print(f"{key}: {value}")

Model Name: fine-tuning-phi-4-mini-onnx-int4-cpu
Model ID: /subscriptions/6415ebd4-1dd7-430f-bd4d-2f5e9419c1cd/resourceGroups/rg-cvi-lab329-h-3/providers/Microsoft.MachineLearningServices/workspaces/cvi-lab329-h-3/models/fine-tuning-phi-4-mini-onnx-int4-cpu/versions/1
Model Version: 1
Model Description: Fine tuning by MSOlive

Model Location: azureml://subscriptions/6415ebd4-1dd7-430f-bd4d-2f5e9419c1cd/resourceGroups/rg-cvi-lab329-h-3/workspaces/cvi-lab329-h-3/datastores/workspaceblobstore/paths/LocalUpload/4e85d7df997d513303aa791ce94b3cc5/onnx

Full Model Properties:
job_name: None
_intellectual_property: None
_system_metadata: None
_is_anonymous: False
_auto_increment_version: False
auto_delete_setting: None
name: fine-tuning-phi-4-mini-onnx-int4-cpu
description: Fine tuning by MSOlive
tags: {}
properties: {}
_print_as_yaml: False
_id: /subscriptions/6415ebd4-1dd7-430f-bd4d-2f5e9419c1cd/resourceGroups/rg-cvi-lab329-h-3/providers/Microsoft.MachineLearningServices/workspaces/cvi-lab329